
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [131]:
import pandas as pd
import numpy as np
from scipy import stats as st

**1.WHAT TEST IS APPROPRIATE FOR THIS PROBLEM?**


** CHI2 TEST IS THE MOST APPROPRIATE TEST **



In [132]:
data = pd.io.stata.read_stata('/Users/sowmyamoka/Satish_Python/Statistics_Projects/statistics project 2/data/us_job_market_discrimination.dta')

In [133]:
# number of callbacks for balck-sounding names
sum(data[data.race=='b'].call)

157.0

In [134]:
data2=pd.DataFrame(data.race,index=np.arange(len(data.race)))

In [135]:
data2['call']=data.call

In [136]:
data2.head(5)# seperating the race and call for analysis

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [137]:
data_aggregate=pd.crosstab(data2.call,data2.race) # CONTINGENCY TABLE OBSERVED

In [138]:
data_aggregate

race,b,w
call,,
0.0,2278,2200
1.0,157,235


In [139]:
data_aggregate=data_aggregate.rename(index={0.0:"NO_CALL",1.0:"CALL"},columns={'b':'BLACK','w':'WHITE'})

* **WHAT ARE THE NULL AND ALTERNATE HYPOTHESIS?**
* **H_NULL: THERE IS DISCRIMINATION**
* **H_NULL: THERE IS NO DISCRIMINATION**


In [140]:
data_observed=data_aggregate

In [141]:
data_observed

race,BLACK,WHITE
call,,
NO_CALL,2278,2200
CALL,157,235


In [142]:
def chi2_pvalue(data_frame):
                overall_pop_ratio=data_frame.apply(lambda x : x.sum(), axis=1)/data_frame.sum().sum()  
            # calculating the ideal ratio
                data_expected=data_frame.apply(lambda x : (x/x)*x.sum(),axis=0).multiply(overall_pop_ratio, axis=0)
            # data frame expected
                chi2_value=(np.square((data_frame)-(data_expected)).div(data_expected)).sum().sum()# chi2 statistic
                DOF=((len(data_frame.index)-1)*(len(data_frame.columns)-1)) # calculating the DOF
                P=(1-stats.chi2.cdf(chi2_value,DOF))# P value
                return P, chi2_value


In [143]:
chi2_pvalue(data_observed)

(3.9838868375885461e-05, 16.879050414270225)

* ** P VALUE LESS THAN 0.05 , SO REJECT THE NULL HYPOTHESIS**

In [144]:
overall_pop_ratio=data_observed.apply(lambda x : x.sum(), axis=1)/data_observed.sum().sum()  
data_expected=data_observed.apply(lambda x : (x/x)*x.sum(),axis=0).multiply(overall_pop_ratio, axis=0)

    

In [145]:
data_expected

race,BLACK,WHITE
call,,
NO_CALL,2239.0,2239.0
CALL,196.0,196.0


In [146]:
population_call_mean=data_expected['BLACK']['CALL']/data_expected['BLACK'].sum()

In [147]:
population_call_mean

0.080492813141683772

In [148]:
population_call_mean_std=(population_call_mean)*(1-population_call_mean)/np.sqrt(2435)

In [149]:
population_call_mean_std

0.0014999015400988711

In [150]:
def confidence_interval(mean,std,percentile):
    z=st.norm.ppf((1+0.01*percentile)/2)
    upper_limit = mean+z*std
    lower_limit= mean-z*std
    print(upper_limit)
    print(lower_limit)
    return 


In [151]:
confidence_interval(population_call_mean,population_call_mean_std,95)

0.0834325661406
0.0775530601427


In [152]:
sample_call_mean_black=data_observed['BLACK']['CALL']/data_observed['BLACK'].sum()

In [153]:
sample_call_mean_black

0.064476386036960986

** THE MEAN IN CALL BACK FOR BLACK IS OUT OF 95% CI FOR POPULATION MEAN**

In [154]:
sample_call_mean_white=data_observed['WHITE']['CALL']/data_observed['WHITE'].sum()

In [155]:
sample_call_mean_white

0.096509240246406572

** THE MEAN IN CALL BACK FOR WHITE IS OUT OF 95% CI FOR POPULATION MEAN**

In [156]:
def P_value_no_diff(P1,P2): # probability of no difference between samples
    P1,P2
    P1_std=P1*(1-P1)/np.sqrt(2435)
    P2_std=P2*(1-P2)/np.sqrt(2435)
    diff_mean_std=np.sqrt((np.square(P1_std))+(np.square(P2_std)))
    diff_mean=np.abs(P1_std-P2_std)
    z=diff_mean/diff_mean_std
    P=1-st.norm.cdf(z)
    return  P1_std

In [157]:
P_value_no_diff(0.064476,0.096509)

0.0012223724052172961

** THE P VALUE IS LESS THAN 0.05 , SO THE HYPOTHESIS THAT THERE IS NO DIFFERENCE IN MEAN FOR RACES IS REJECTED**

** FROM CHI2 TEST THE PROBABILITY OF MEANS BEING THE SAME IS VERY LOW AND CLOSE TO ZERO**

** THERE IS A CLEAR DIVISION IN THE CALL BACK FOR BOTH THE RACES**